In [4]:
import os 
from Bio import SeqIO
import pandas as pd


> Make the multifasta domain 

In [13]:
path_other = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea"

df_other_info = pd.read_csv(f"{path_other}/full_other_bea.info.tsv", header = 0, sep = "\t")
df_other_info["clean_accession"] = df_other_info["accession"].apply(lambda x : x.upper())
dico_seq = {record.description.split(".")[0]: record.seq for record in SeqIO.parse(f"{path_other}/full_other_bea.multi.fasta", "fasta")}


In [9]:
dico_seq

{'AUV61507': Seq('MDQDTKTIIQYPTSGDEYDIPFDYLSRKFVRVSLVSNTQRVLLDNITDYRYVSR...LNA'),
 'BAN78446': Seq('MGYFQMTRNVEEIFGGVVVAPHQIPFKYTSTNGGETFLSLPFYPITGFVTINSG...EEK'),
 'AZF89844': Seq('MALYRQGKAAMDANGVITGTGTKWQSSLSLIRPGSTIMFLSSPIQMAVINKVVS...EVL'),
 'ALT58497': Seq('MDQDIKTVIQYPVGATEFDIPFDYLSRKFVRVSLVTDENRRLLSNITEYRYVSK...GEQ'),
 'ALT58498': Seq('MLNNLNQPKGSTIGVLKDGRTIQQAIDGLENPVHYVKDVSITPSALLAVAVEAA...IGP'),
 'APZ82768': Seq('MALVDLVRAGGYSIEYPQFSSMAKLKEFPHSEDGKLVRLLSWHEGVGLGGGLFK...VVK'),
 'QTP95996': Seq('MALYREGKAAMAADGTVTGTGTKWQSSLSLIRPGATIMFLSSPIQMAVVNKVVS...MSS'),
 'YP_003347555': Seq('MDQDIKTIIQYPTGATEFDIPFDYLSRKFVRVSLVSDDNRRLLRNITEYRYVSK...TYK'),
 'YP_003347556': Seq('MLDNFNQPKGSTIGVLKDGRTIQEAFDSLPRLESFSGSTATDKLRAAITLGVSE...FVS')}

In [14]:
df_other_info

,Phage,accession,boundaries,clean_accession
0,Klebsiella phage SH-Kp 152410,AUV61507,305-917,AUV61507
1,klebsiella_phage_0507_kn2_1,ban78446,304-1003,BAN78446
2,Klebsiella phage 13,azf89844,294-571,AZF89844
3,Klebsiella phage vB_KpnP_IME205,ALT58497,333-657,ALT58497
4,Klebsiella phage vB_KpnP_IME205,ALT58498,59-596,ALT58498
5,Klebsiella phage vB_KpnP_KpV74,APZ82768,116-426,APZ82768
6,Klebsiella phage B1,qtp95996,300-648,QTP95996
7,Klebsiella phage KP32,YP_003347555,334-765,YP_003347555
8,Klebsiella phage KP32,YP_003347556,53-359,YP_003347556
9,Klebsiella phage KP32,YP_003347556,53-487,YP_003347556


In [15]:
with open(f"{path_other}/depo_other_bea.multi.fasta", "w") as outfile : 
    for _, row in df_other_info.iterrows() : 
        start, end = row["boundaries"].split("-")[0] , row["boundaries"].split("-")[1]
        seq_full = dico_seq[row["clean_accession"]]
        dpo_domain = seq_full[int(start) : int(end)]
        outfile.write(f">{row['clean_accession']}__{start}_{end}\n{dpo_domain}\n")
        

> Move to server : 

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea/depo_other_bea.multi.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/


rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project/depo_other_bea.multi.fasta.esm2_out \
/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea 

> Make DF : 

In [18]:
import torch
import os 
import pandas as pd
from tqdm import tqdm 


path_esm = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea/depo_other_bea.multi.fasta.esm2_out"

embeddings_esm = {}
for file in tqdm(os.listdir(path_esm)) :
    path_file = f"{path_esm}/{file}"
    index = file.split(".")[0]
    embb = torch.load(f"{path_file}")["mean_representations"][33].tolist()
    embeddings_esm[index] = embb
    
with open(f"/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea/depo_other_bea.esm2.embedding.csv" , "w") as outfile :
    for index in tqdm(embeddings_esm) :
        outfile.write(f"{index},")
        #for _,  emb in enumerate(embeddings_esm[index]) :
        #    outfile.write(f"{emb},")
        data_emb = ",".join([str(item) for item in embeddings_esm[index]])
        outfile.write(f"{data_emb}\n")
        #outfile.write("\n")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1580.31it/s]


In [21]:
other_bea_df = pd.read_csv(f"{path_other}/depo_other_bea.esm2.embedding.csv", header = None, sep = ",", index_col = 0)

In [22]:
other_bea_df

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
ALT58497__333_657,-0.010812,0.034124,0.065600,-0.009714,-0.112024,0.025193,0.053057,-0.117283,0.079996,0.034734,...,0.001143,-0.001576,0.100628,-0.017003,0.007330,0.052329,-0.005884,-0.053916,0.008181,0.090729
AZF89844__294_571,0.017776,0.075359,-0.009375,-0.029603,-0.082510,0.028871,-0.012899,-0.047576,0.084600,0.010918,...,0.004880,0.026317,0.121081,0.003259,0.040123,0.059707,0.035005,-0.070696,0.089692,0.072651
QTP95996__300_648,0.040388,0.080415,-0.004000,-0.030825,-0.084768,-0.003406,0.017062,-0.046310,0.079506,-0.000662,...,0.012946,0.022542,0.090343,0.000989,0.020482,0.080367,0.030350,-0.062859,0.031516,0.055045
AUV61507__305_917,0.008883,0.041041,-0.009967,0.004375,-0.135688,0.072138,0.012696,-0.087694,0.086344,0.021472,...,0.053671,0.032256,0.100796,0.029916,0.021635,0.010553,0.033686,-0.087905,0.065211,0.022509
APZ82768__116_426,0.001793,0.043011,0.002497,-0.026185,-0.073375,0.044104,-0.002758,-0.063913,0.100008,0.089924,...,-0.018892,0.052273,0.117460,-0.016095,0.023953,0.000176,0.024056,-0.090518,0.057145,0.084150
YP_003347556__53_487,0.001579,0.021468,0.003672,-0.055815,-0.085401,-0.000380,0.063201,-0.080216,0.096091,0.014615,...,-0.029344,0.000608,0.082081,-0.007823,0.033662,0.016347,0.077357,-0.064431,0.023578,0.106990
YP_003347555__334_765,0.015444,0.059503,-0.012123,-0.025685,-0.089604,-0.003112,0.042158,-0.041153,0.090919,0.041549,...,-0.024758,-0.000803,0.093412,0.009340,0.055455,0.022576,0.067360,-0.070687,0.059636,0.075206
YP_003347556__53_359,0.000372,0.033700,0.031151,-0.063508,-0.079455,0.013988,0.050378,-0.051854,0.100425,0.029021,...,-0.030940,0.003717,0.095932,-0.006771,0.032133,0.018732,0.043351,-0.075403,0.046667,0.130569
BAN78446__304_1003,0.020276,0.044222,-0.006038,0.009521,-0.126313,0.019006,0.006778,-0.077483,0.070887,0.045317,...,0.026894,-0.011350,0.100422,0.033347,0.029296,0.037818,0.027805,-0.065334,0.035186,0.025852
